In [1]:
import pandas as pd

df = pd.read_csv("intents.csv")
df


,intent,text
0,greeting,Hi
1,greeting,Hello
2,greeting,Hey
3,greeting,Hi there
4,greeting,Good morning
5,greeting,Good evening
6,greeting,Hey buddy
7,greeting,Hello chatbot
8,goodbye,Bye
9,goodbye,Goodbye


In [2]:
responses = {
    "greeting": "Hello! How can I help you?",
    "goodbye": "Goodbye! Have a nice day.",
    "thanks": "You're welcome!",
    "about_ai": "Artificial Intelligence is the simulation of human intelligence in machines.",
    "about_chatbot": "I am a simple AI-powered chatbot designed to answer basic questions."
}


In [3]:
def detect_intent(user_input):
    user_input = user_input.lower()
    
    if any(word in user_input for word in ["hi", "hello", "hey"]):
        return "greeting"
    elif any(word in user_input for word in ["bye", "goodbye", "see you"]):
        return "goodbye"
    elif any(word in user_input for word in ["thanks", "thank you"]):
        return "thanks"
    elif "artificial intelligence" in user_input or "ai" in user_input:
        return "about_ai"
    elif "chatbot" in user_input:
        return "about_chatbot"
    else:
        return "fallback"


In [4]:
def chatbot_response(user_input):
    intent = detect_intent(user_input)
    return responses.get(intent, "Sorry, I didn't understand that.")


In [5]:
print("Chatbot is running. Type 'exit' to stop.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Bot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print("Bot:", response)


Chatbot is running. Type 'exit' to stop.


You:  exit


Bot: Goodbye!


In [6]:
import pandas as pd

df = pd.read_csv("intents.csv")
df


,intent,text
0,greeting,Hi
1,greeting,Hello
2,greeting,Hey
3,greeting,Hi there
4,greeting,Good morning
5,greeting,Good evening
6,greeting,Hey buddy
7,greeting,Hello chatbot
8,goodbye,Bye
9,goodbye,Goodbye


In [7]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df['clean_text'] = df['text'].apply(clean_text)
df.head()


,intent,text,clean_text
0,greeting,Hi,hi
1,greeting,Hello,hello
2,greeting,Hey,hey
3,greeting,Hi there,hi there
4,greeting,Good morning,good morning


In [8]:
X = df['clean_text']
y = df['intent']


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
X_tfidf = tfidf.fit_transform(X)


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


In [11]:
from sklearn.naive_bayes import MultinomialNB

intent_model = MultinomialNB()
intent_model.fit(X_train, y_train)


,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [12]:
from sklearn.metrics import accuracy_score

y_pred = intent_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8


In [13]:
def predict_intent_ml(message):
    message_clean = clean_text(message)
    message_tfidf = tfidf.transform([message_clean])
    return intent_model.predict(message_tfidf)[0]


In [14]:
print(predict_intent_ml("Hi there"))
print(predict_intent_ml("What is artificial intelligence"))
print(predict_intent_ml("Thanks a lot"))


greeting
about_ai
thanks


In [17]:
def predict_intent_ml(message):
    message_clean = clean_text(message)
    message_tfidf = tfidf.transform([message_clean])
    return intent_model.predict(message_tfidf)[0]


In [18]:
'''brain of the bot'''
def detect_intent_hybrid(user_input):
    rule_intent = detect_intent_rule(user_input)
    if rule_intent:
        return rule_intent
    else:
        return predict_intent_ml(user_input)


In [19]:
def chatbot_response(user_input):
    intent = detect_intent_hybrid(user_input)
    return responses.get(intent, responses["fallback"])


In [23]:
print("Hybrid Chatbot running. Type 'exit' to stop.")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:

        print("Bot:", responses["goodbye"])
        break
    print("Bot:", chatbot_response(user_input))


Hybrid Chatbot running. Type 'exit' to stop.


You:  hi


Bot: Hello! How can I assist you today?


You:  what do you do


Bot: Goodbye! Have a great day.


You:  exit


Bot: Goodbye! Have a great day.


In [ ]:

## Limitations
- The chatbot's ML-based intent classification depends on the size and quality of the training dataset.
- With limited intent examples, some user inputs may be misclassified.
- The chatbot currently supports only text-based interaction.
- Context handling is minimal and does not persist across long conversations.

In [ ]:

## Future Scope
- The intent dataset can be expanded to improve classification accuracy.
- Advanced NLP models such as deep learning can be integrated.
- Context-aware conversation handling can be implemented.
- The chatbot can be deployed as a web or mobile application.


In [ ]:
## Conclusion
The AI-powered chatbot successfully interacts with users using a hybrid architecture that combines rule-based logic and machine learning. 
The system demonstrates how conversational AI can be built efficiently using NLP techniques and supervised learning models.
